<a href="https://colab.research.google.com/github/b62mp1/ExistingResults/blob/main/Lstm_1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

     |████████████████████████████████| 6.4 MB 7.3 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

import keras
import tensorflow as tf
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense
import plotly.graph_objects as go


In [ ]:
stock_name = "GOOG"
dataset=pdr.get_data_yahoo(stock_name, start='2015-01-01', end='2018-03-08').reset_index()
dataset.index = dataset['Date']

print("Number of rows and columns:", dataset.shape)
dataset.head(5)

[*********************100%***********************]  1 of 1 completed
Number of rows and columns: (800, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2015-01-02,2015-01-02,527.561584,529.815369,522.665039,523.373108,523.373108,1447563
2015-01-05,2015-01-05,521.827332,522.894409,511.655243,512.463013,512.463013,2059840
2015-01-06,2015-01-06,513.589966,514.761719,499.678131,500.585632,500.585632,2899940
2015-01-07,2015-01-07,505.611847,505.855164,498.281952,499.727997,499.727997,2065054
2015-01-08,2015-01-08,496.626526,502.101471,489.655640,501.303680,501.303680,3353582


In [ ]:
close_data = dataset['Close'].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = dataset['Date'][:split]
date_test = dataset['Date'][split:]

In [ ]:
look_back = 15

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [35]:
model = Sequential()
model.add(
    LSTM(1000,
        activation='relu',
        input_shape=(look_back,1))
)

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 25
model.fit(train_generator,epochs=num_epochs, verbose=1)

Epoch 1/25
32/32 [==============================] - 18s 513ms/step - loss: 58005.6523
Epoch 2/25
32/32 [==============================] - 12s 387ms/step - loss: 14306.8672
Epoch 3/25
32/32 [==============================] - 13s 391ms/step - loss: 4839.5454
Epoch 4/25
32/32 [==============================] - 13s 392ms/step - loss: 1720.3922
Epoch 5/25
32/32 [==============================] - 12s 389ms/step - loss: 3977.3396
Epoch 6/25
32/32 [==============================] - 12s 387ms/step - loss: 2561.1987
Epoch 7/25
32/32 [==============================] - 13s 410ms/step - loss: 3014.4346
Epoch 8/25
32/32 [==============================] - 14s 421ms/step - loss: 1573.6564
Epoch 9/25
32/32 [==============================] - 12s 389ms/step - loss: 2300.4080
Epoch 10/25
32/32 [==============================] - 12s 385ms/step - loss: 1920.6206
Epoch 11/25
32/32 [==============================] - 12s 389ms/step - loss: 1780.3424
Epoch 12/25
32/32 [==============================] - 12s 387m

In [37]:
prediction = model.predict(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Predicted Data'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Test Data'
)
layout = go.Layout(
    title = "Google Stock",
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()